# Importing Libraries

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama
from ollama import chat

# Website class to scrape website data

In [2]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """

    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [3]:
commerce = Website(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/channel-setup-retail"
)
print(commerce.title)

Set up a retail  channel - Commerce | Dynamics 365 | Microsoft Learn


# System prompt for the model

In [4]:
system_prompt = """
You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related.
Respond in markdown.
"""

# Function to create user prompt for the model

In [5]:
def user_prompt_for(Website):
    user_prompt = f"the Website title is {Website.title}"
    user_prompt += """
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n
    """
    user_prompt += Website.text
    return user_prompt

# Function to summarize the content of a website

In [6]:
def summarize(url):
    website = Website(url)
    title = website.title  # Assuming the Website object has a title attribute
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)},
    ]
    # Disable streaming, get the entire response in one go
    response = ollama.chat(
        model="chevalblanc/gpt-4o-mini", messages=messages, stream=False
    )
    summary_content = response["message"]["content"]
    return {"title": title, "summary": summary_content}

# Function to display the summary of a website

In [7]:
def display_summary(url):
    summary = summarize(url)
    # Format the title and summary separately for Markdown
    title = f"# {summary['title']}\n"  # Markdown header for the title
    content = summary["summary"]  # Main content
    display(Markdown(title + "\n" + content))

# Example usage

In [8]:
display_summary(
    "https://learn.microsoft.com/en-us/dynamics365/commerce/dev-itpro/store-commerce"
)

# Store Commerce app - Commerce | Dynamics 365 | Microsoft Learn

The text in the image appears to be from a user manual or guide for configuring and troubleshooting Store Commerce, which is an app used with Dynamics 365 Commerce. The instructions provided cover how to configure deployment options such as Remote app content during installation, activate the application using guided activation, troubleshoot setup issues, customize features through the commerce SDK, integrate hardware devices, manage WebView2 versions on Windows, and provide known issue information about the Microsoft Edge WebView2 control.

The text also provides links for further reading or additional resources related to Store Commerce. These include a list of capabilities provided by the app, an article outlining modernizing technology stack with Dynamics 365 Commerce in-store technology, technical talk series on commerce SDKs, and feedback options through which users can provide input about their experience using the application.

Overall, this text appears to be part of documentation that provides instructions for configuring and troubleshooting Store Commerce within a larger system or platform.